In [833]:
import requests
from bs4 import BeautifulSoup
import time 
import random 
import re
from fake_useragent import UserAgent
import datetime
from elasticsearch import Elasticsearch
from tqdm import tqdm
import string
es=Elasticsearch()

In [834]:
punct = string.punctuation + '«»—…“”'

In [809]:
class KanobuArticle:
    def __init__(self):
        self.date=""
        self.time=""
        self.words=""
        self.rubr=""
        self.hashtag=""
        self.url=""
        self.author=""
        self.head=""
        self.corona=""
        self.text=""

        

    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "words":"'+self.words+'", "rubrics":"'+self.rubr+'", "hashtag":"'+self.hashtag+'", "title":"'
        +self.head+'", "link":"'+self.url+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'", "corona":"'+self.corona+'"}'
        return res


    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "hashtag":self.hashtag, \
             "title":self.head, "words":self.words, "link":self.url, "author":self.author,
             "corona":self.corona, "text":self.text.replace('"', '\\"')}
        return res

In [810]:
def GetPagesArticles(number_of_pages):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    url = 'https://kanobu.ru/news/?page='
    links = []
    for page in range(1, number_of_pages+1):
        r = requests.get(url + str(page), headers = headers)
        time.sleep(random.randint(1,6))
        content = r.content
        html = content.decode('UTF-8')
        soup = BeautifulSoup(html)
        
        for link in soup.find_all('a', attrs={'class':'c-item_foot d-b t-l'}):
            links.append(link.get('href'))
    
    return links    

In [811]:
def clean_text (text):
    text = text.replace('\xa0', ' ')
    text = text.replace ('\n', ' ')
    text = text.replace ('  ', ' ')
    return text

In [812]:
def add_article(link):
    art = KanobuArticle()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    page = requests.get(link, headers=headers)
    content = page.content
    html = content.decode('UTF-8')
    soup = BeautifulSoup(html, features="lxml")

    date = soup.find('div', attrs={'class': 'c-title_footer'}).get_text()
    art.date = date.split(', ')[0]
    art.time = date.split(', ')[1]
    
    author = soup.find('a', attrs={'class': 'c-title_author'}).get_text()
    art.author = author

    text = soup.find('div', attrs={'class': 'c-detail_content'}).get_text()
    art.text = clean_text(text)

    words = len(clean_text(text).split())
    art.words = words
    
    art.corona = corona_detector(art.text)

    title = soup.find('h1', attrs={'class': 'c-title_body'}).get_text()
    art.head = clean_text(title)
    
    art.url = link
    
    rubric = soup.find('div', attrs={'class': 'c-title_head t-t-u'}).get_text()
    r = rubric.split(' | ')
    art.rubr = r[0]
    
    try:
        art.hashtag = r[1]
    except:
        pass
    
    # print(art)
    # print(text)
    # print(date)
    # print(author)
    print(title)
    #print(words)
    # print(r[0])
    # print(r[1])

    res = es.search(index="kanobu", body={"query": {"match_all": {}}})
    maxid = res['hits']['total']['value']
    es.index(index="kanobu", doc_type='article', id=maxid, body=art.toDict())


In [813]:
covid_words = {"коронавирус", "коронавируса", "коронавирусом", "коронавирусе","коронавирусу", 
               "коронавирусная", "коронавирусный", "коронавирусное",
               "вирус", "вируса", "вирусом", "вирусы", "вирусе", "вирусу",
               "инфекция", "инфекции", "инфекцией", "инфекцию",
               "эпидемия", "эпидемии", "эпидемию","эпидемией",
               "пандемия", "пандемии", "пандемию", "пандемией", 
               "карантин", "карантину", "карантина", "карантином",
               "самоизоляция", "самоизоляции", "самоизоляцией", "covid-19", "covid",
               "corona"}

In [814]:
def corona_detector(text):
    words = [word.strip(string.punctuation).lower() for word in text.split()]
    counter = 0
    for word in words:
        if word in covid_words:
            counter+=1
    return counter

In [815]:
def create_db(index):
    es.indices.create(index=index)
    mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":" HH:mm"},
                                "words":{"type":"double"},
                                "hashtag":{"type":"text","analyzer" : "russian"},
                                "rubrics":{"type":"text","analyzer" : "russian"},
                                "corona":{"type":"double"},
                                "url":{"type":"text","analyzer" : "russian"},
                                "text":{"type":"text","analyzer" : "russian"},
                                "title":{"type":"text","analyzer" : "russian"}}}}
    es.indices.put_mapping(index=index, doc_type='article', body=mapit, include_type_name=True)

In [816]:
def GetArticles(links):
    l = 'https://kanobu.ru'
    create_db("kanobu")
    for link in links:
        add_article(l+link)
        time.sleep(random.randint(1,3))

In [819]:
es.indices.delete(index="kanobu") 

{'acknowledged': True}

In [820]:
GetArticles(GetPagesArticles(1))

Криповое «Поле чудес», уроки русского от Миллы Йовович и кошачьи анусы: 10 главных новостей недели
Минкульт предлагает ужесточить борьбу с пиратами — больше штрафовать и быстрее удалять
Джей Джей Абрамс и Epic Games сделали мобильную игру про шпионов. Вот первые скриншоты
Актеры GTA V призвали не верить инсайдам о GTA VI: «Откуда они берут это?»
Гоблин правильно озвучил художественный фильм «Джентльмены» для «Ростелекома»
Звезда «Достать ножи» советует «Надеть маски». Джейми Ли Кертис рекомендует сидеть на карантине
ГеймНет-челлендж: вебинар с организаторами
Инсайдер рассказал про Resident Evil 8. Изначально это была Revelations 3
Российский художник показал постапокалипсис с курьерами в главной роли
IGN проведет мероприятие, которое заменит нам E3 2020
«Человека-муравья 3» напишет сценарист «Рика и Морти»
Режиссер «Хищных птиц» высказалась о неудачах фильма
Саша Грей постримила Resident Evil 3: Remake
«Это Веном?» — в интернете считают, что нашли реального симбиота
Новый выпуск «Поле 

### Интерфейс

In [821]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

In [835]:
text1 = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

menu = widgets.Dropdown(
       options=['id', 'title', 'date', 'author', 'hashtag', 'rubrics', 'corona'],
       value='id',
       description='How to find?')

text = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

text2 = widgets.Text(
    value='',
    placeholder='Enter the link of the article to add',
    description='Link:',
    disabled=False
)

order = widgets.Dropdown(
       options=['ascending', 'descending'],
       value='ascending',
       description='Order')

sortby = widgets.Dropdown(
       options=['_id', 'words', 'corona'],
       value='_id',
       description='Sort by')

number = widgets.IntSlider(
    value=10,
    min=1,
    max=10000,
    step=1,
    description='How many articles?',
)

counter = widgets.Dropdown(
       options=['min', 'max', 'avg'],
       value='max',
       description='measure')

counter2 = widgets.Dropdown(
       options=['words', 'corona'],
       value='words',
       description='counter')

text3 = widgets.Text(
    value='',
    placeholder='Enter article id to delete',
    description='Id:',
    disabled=False
)

id_upd = widgets.Text(
    value='',
    placeholder='Enter your id',
    description='Text:',
    disabled=False
)

menu_upd = widgets.Dropdown(
       options=['title', 'date', 'author', 'hashtag', 'rubrics', 'text', 'corona', 'time'],
       value='title',
       description='Update')

text_upd = widgets.Text(
    value='',
    placeholder='Enter your update',
    description='Update:',
    disabled=False
)

In [831]:
button1 = widgets.Button(description='Count')
out = widgets.Output()

def on_button_clicked1(_):
    with out:
        clear_output()
        res=es.search(index = "kanobu", body = {"size": 0, "aggs": {counter.value: {counter.value: {"field": counter2.value }}}})
        print(round(res["aggregations"][counter.value]["value"]))
        

button1.on_click(on_button_clicked1)

In [824]:
button2 = widgets.Button(description='Match with options')
out = widgets.Output()

def on_button_clicked2(_):
    with out:
        clear_output()
        if menu.value == 'id':
            id_search = int(text.value)
            print(es.get(index="kanobu", doc_type='article', id=id_search))
        
        else:
            res = es.search(index="kanobu", body={"query": {"match": {menu.value : text.value}}}, size=100)
            for part in res["hits"]["hits"]:
                print(part)

                
button2.on_click(on_button_clicked2)

In [825]:
button3 = widgets.Button(description='Add article')
out = widgets.Output()

def on_button_clicked3(_):
    with out:
        clear_output()
        link = text2.value
        add_article(link)

        
button3.on_click(on_button_clicked3)

In [826]:
button4 = widgets.Button(description='Delete article')
out = widgets.Output()

def on_button_clicked4(_):
    with out:
        clear_output()
        id_del = int(text3.value)
        es.delete(index="kanobu",doc_type="article",id=id_del)
        print('Done')

        
button4.on_click(on_button_clicked4)

In [827]:
button5 = widgets.Button(description='Sort articles')
out = widgets.Output()


def on_button_clicked5(_):
    with out:
        clear_output()
        if order.value == 'ascending':
            ord = str(sortby.value)+':asc'
        elif order.value == 'descending':
            ord = str(sortby.value)+':desc'
        res = es.search(index="kanobu", body={"query": {"match_all": {}}}, sort=ord,size=number.value)
        for part in res["hits"]["hits"]:
            print(part)
            
            
button5.on_click(on_button_clicked5)

In [828]:
button_upd = widgets.Button(description='Update')
out = widgets.Output()


def on_button_clicked_upd(_):
    with out:
        clear_output()
        your_id = int(id_upd.value)
        upd_area = str(menu_upd.value)
        text = str(text_upd.value)
        es.update(index='kanobu', doc_type='article', id=your_id, body={"doc": {upd_area: text}})
        print('Done')

        
button_upd.on_click(on_button_clicked_upd)

In [829]:
def on_button_clicked_group(_):
    with out:
        clear_output()
        res = es.search(index="kanobu", body={"size":number_size.value, "query": {"match_all": {}}, "collapse": {"field" : "rubrics_id", "inner_hits": {"name":"rubrics_id","size": number_in_group.value}}})
        for group in res['hits']['hits']:
            print(group)

In [836]:
box1 = widgets.VBox([menu, text, button2])
box2 = widgets.VBox([text2, button3, text3, button4])
box3 = widgets.VBox([id_upd, menu_upd, text_upd, button_upd])
box4 = widgets.VBox([sortby,order, number, button5])
box5 = widgets.VBox([counter2, counter, button1])
widgets.VBox([box1, box2, box3, box4, box5, out])